<a href="https://colab.research.google.com/github/Esantomi/gaebyeok-scraper/blob/main/gaebyeok_scraper_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 『개벽(開闢)』 전문 스크래퍼 v.0.3
### 개요
- 출처 : [한국사데이터베이스 - 한국근현대잡지자료](http://db.history.go.kr/item/level.do?sort=levelId&dir=ASC&start=1&limit=77&page=1&pre_page=0&setId=-1&totalCount=0&prevPage=1&prevLimit=20&itemId=ma&types=&synonym=off&chinessChar=on&brokerPagingInfo=&levelId=ma_013)
- 목적 : 『개벽(開闢)』의 모든 텍스트와 메타데이터를 excel로 저장
- 제작 : [지해인](https://github.com/Esantomi)

### URL 규칙
- `http://db.history.go.kr/item/level.do?sort=levelId&dir=ASC&start=4&limit=77&page=1&pre_page=0&setId=-1&totalCount=0&prevPage=4&prevLimit=&itemId=ma&types=&synonym=off&chinessChar=on&brokerPagingInfo=&levelId=ma_013_'` + 호(volume) 넘버 4자리_텍스트 넘버 4자리
  - 개벽 제28호는 1부, 2부로 나뉘며, 호 넘버도 각각 271, 272이므로 별도 처리함.
  - 호 처음에 실린 글은 텍스트 넘버가 0 또는 1인 경우가 있으니 별도 처리함.
  - 호 마지막에 실린 사고·편집후기는 텍스트 넘버 끝자리가 1인 경우가 있으니 별도 처리함.
- div 태그의 `#cont_view`에 본문 삽입됨.
- metadata는 tbody > tr > td 안에 있고, tbody는 4개의 tr(순서대로 잡지명, 발행일, 기사제목, 기사형태)을 가짐
  - metadata가 잡지명, 발행일, 기사제목, **필자**, 기사형태의 다섯 요소를 갖는 경우도 있으므로, 이에 대한 대처가 필요함. (v0.3)

### 참고
- `connection reset by peer` error가 뜰 경우, 적당히 `time.sleep(round(random.uniform(3, 5), 2))` 삽입 요망.

# 모듈 import 및 함수 정의
- 모듈 및 `get_URL` 함수는 반드시 선언하고 스크래핑에 들어가야 함
- 이후 **본문 + 메타데이터 스크래핑 코드**를 돌릴 것

In [ ]:
# coding: utf-8
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd

In [ ]:
# 404 error 여부 확인용

# test_URL = 'http://db.history.go.kr/item/level.do?sort=levelId&dir=ASC&start=4&limit=77&page=1&pre_page=0&setId=-1&totalCount=0&prevPage=4&prevLimit=&itemId=ma&types=&synonym=off&chinessChar=on&brokerPagingInfo=&levelId=ma_013_0010_1010'
# response = requests.get(test_URL)
# response.status_code

In [ ]:
# URL 리턴 함수

def get_URL(vol_number, id_number):
  return 'http://db.history.go.kr/item/level.do?sort=levelId&dir=ASC&start=4&limit=77&page=1&pre_page=0&setId=-1&totalCount=0&prevPage=4&prevLimit=&itemId=ma&types=&synonym=off&chinessChar=on&brokerPagingInfo=&levelId=ma_013_' + str(vol_number).zfill(4) + '_' + str(id_number).zfill(4)

# 개벽 본문 스크래핑
- 메타데이터 없이 본문만 스크래핑하는 코드
- 본문 예시
  ![image](https://user-images.githubusercontent.com/61646760/165749909-d839f166-4625-48a9-ab80-90243a8c6dc1.png)

In [ ]:
from itertools import chain # 두 번째 for문 0, 1 처리용

# 개벽 전체 텍스트
whole_req = ""

# 개벽 제28호(1부, 2부) 처리용
volumes = list(range(10, 760, 10))
volumes[27:27] = [271, 272]

# 개벽 전체 텍스트 스크래핑
for vol_number in volumes:  # for vol_number in range(10, 760, 10):

  # 호(volume) 내부 텍스트 스크래핑
  for id_number in chain([0, 1], range(10, 9999, 10)):
    URL = get_URL(vol_number, id_number)
    response = requests.get(URL)

    # 404 error 발생 시 다음 volume으로 이동
    if response.status_code == 404:
      if id_number > 10:
        break
    else:
      # URL을 get하여 얻은 response의 text를 리스트에 삽입
      whole_req = whole_req + response.text
    
      # 마지막 텍스트 여부 확인용
      last_success = id_number

    print(vol_number, id_number)  # 확인용

  # 호(volume) 마지막 텍스트 스크래핑용 (1로 끝나는 경우 대비)
  URL = get_URL(vol_number, last_success + 1)
  response = requests.get(URL)
  if response.status_code != 404:
    whole_req = whole_req + response.text
    print(vol_number, last_success + 1)  # 확인용

# parsing 및 텍스트 내용 검색
soup = bs(whole_req, 'html.parser')
elements = soup.select('#cont_view')  # 텍스트 내용

# 본문 list
txt = []

# 본문 list에 enumerate로 내용 넣기
for index, element in enumerate(elements, 1):
  txt.append(element.text)

10 0
10 1
10 10
10 20
10 30
10 40
10 50
10 60
10 70
10 80
10 90
10 100
10 110
10 120
10 130
10 140
10 150
10 160
10 170
10 180
10 190
10 200
10 210
10 220
10 230
10 240
10 250
10 260
10 270
10 280
10 290
10 300
10 310
10 320
10 330
10 340
10 350
10 360
10 370
10 380
10 390
10 400
10 410
10 420
10 430
10 440
10 450
10 460
10 470
10 480
20 0
20 1
20 10
20 20
20 30
20 40
20 50
20 60
20 70
20 80
20 90
20 100
20 110
20 120
20 130
20 140
20 150
20 160
20 170
20 180
20 190
20 200
20 210
20 220
20 230
20 240
20 250
20 260
20 270
20 280
20 290
20 300
20 310
20 320
20 330
20 340
20 350
20 360
20 370
20 380
20 390
20 400
20 401
30 0
30 1
30 10
30 20
30 30
30 40
30 50
30 60
30 70
30 80
30 90
30 100
30 110
30 120
30 130
30 140
30 150
30 160
30 170
30 180
30 190
30 200
30 210
30 220
30 230
30 240
30 250
30 260
30 270
30 280
30 290
30 300
30 310
30 320
30 330
30 340
30 350
30 360
30 370
30 380
30 390
30 400
30 410
30 420
30 421
40 0
40 1
40 10
40 20
40 30
40 40
40 50
40 60
40 70
40 80
40 90
40 100
40

# 개벽 메타데이터 스크래핑
- 메타데이터만 스크래핑하는 코드
- 메타데이터 예시
  ![image](https://user-images.githubusercontent.com/61646760/165749779-dce2b1b7-5d86-4f13-864c-42106c1fce3c.png)

## 잡지명, 발행일, 기사제목 추출용

In [ ]:
from itertools import chain # 두 번째 for문 0, 1 처리용

# 개벽 전체 텍스트
whole_req = ""

# 개벽 제28호(1부, 2부) 처리용
volumes = list(range(10, 760, 10))
volumes[27:27] = [271, 272]

# 개벽 전체 텍스트 스크래핑
for vol_number in volumes:  # for vol_number in range(10, 760, 10):

  # 호(volume) 내부 텍스트 스크래핑
  for id_number in chain([0, 1], range(10, 9999, 10)):
    URL = get_URL(vol_number, id_number)
    response = requests.get(URL)

    # 404 error 발생 시 다음 volume으로 이동
    if response.status_code == 404:
      if id_number > 10:
        break
    else:
      # URL을 get하여 얻은 response의 text를 리스트에 삽입
      whole_req = whole_req + response.text
    
      # 마지막 텍스트 여부 확인용
      last_success = id_number

    print(vol_number, id_number)  # 확인용

  # 호(volume) 마지막 텍스트 스크래핑용 (1로 끝나는 경우 대비)
  URL = get_URL(vol_number, last_success + 1)
  response = requests.get(URL)
  if response.status_code != 404:
    whole_req = whole_req + response.text
    print(vol_number, last_success + 1)  # 확인용

### 여기까지는 위 셀의 코드와 동일함!! ###

# parsing 및 텍스트 내용 검색
soup = bs(whole_req, 'html.parser')

# metadata는 tbody > tr > td 안에 있고, tbody는 4개의 tr(순서대로 잡지명, 발행일, 기사제목, (필자), 기사형태)을 가짐
name_of_megazine_req = soup.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(1) > td')  # 잡지명
date_of_issue_req = soup.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(2) > td')     # 발행일
title_of_article_req = soup.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(3) > td')  # 기사제목

# metadata 각각의 list
name_of_megazine = []
date_of_issue = []
title_of_article = []

# list에 내용 넣기 
for element in name_of_megazine_req:
  # print(type(element))  # element는 <class 'bs4.element.Tag'> 자료형
  name_of_megazine.append(element.text)
for element in date_of_issue_req:
  date_of_issue.append(element.text)
for element in title_of_article_req:
  title_of_article.append(element.text)

10 0
10 1
10 10
10 20
10 30
10 40
10 50
10 60
10 70
[<td>사고·편집후기  
							</td>, <td>시  
							</td>, <td>사고·편집후기  
							</td>, <td>논설  
							</td>, <td>李敦化  
							</td>, <td>잔물  
							</td>, <td>朴達成  
							</td>, <td>시  
							</td>]
[<td>세태비평  
							</td>, <td>시  
							</td>, <td>논설  
							</td>]
[<td>사고·편집후기  
							</td>, <td>시  
							</td>, <td>사고·편집후기  
							</td>, <td>논설  
							</td>, <td>李敦化  
							</td>, <td>잔물  
							</td>, <td>朴達成  
							</td>, <td>시  
							</td>, <td>세태비평  
							</td>, <td>시  
							</td>, <td>논설  
							</td>]


## 필자, 기사형태 추출용
- 필자 데이터는 있는 경우도 있고 없는 경우도 있음.
  - 필자 데이터가 있는 경우 : 필자 데이터가 tr:nth-of-type(4)에, 기사형태 데이터가 tr:nth-of-type(5)에 삽입됨.
  - 필자 데이터가 없는 경우 : 기사형태 데이터가 tr:nth-of-type(5)에 삽입됨.
- 이에 따른 로직 작성 필요. 

In [ ]:
from itertools import chain # 두 번째 for문 0, 1 처리용

whole_req = []

# 개벽 제28호(1부, 2부) 처리용
volumes = list(range(10, 760, 10))
volumes[27:27] = [271, 272]

# 개벽 전체 텍스트 스크래핑
for vol_number in volumes:  # for vol_number in range(10, 760, 10):

  # 호(volume) 내부 텍스트 스크래핑
  for id_number in chain([0, 1], range(10, 9999, 10)):
    URL = get_URL(vol_number, id_number)
    response = requests.get(URL)

    # 404 error 발생 시 다음 volume으로 이동
    if response.status_code == 404:
      if id_number > 10:
        break
    else:
      # URL을 get하여 얻은 response의 text를 리스트에 삽입
      whole_req.append(response.text)
    
      # 마지막 텍스트 여부 확인용
      last_success = id_number

    print(vol_number, id_number)  # 확인용

  # 호(volume) 마지막 텍스트 스크래핑용 (1로 끝나는 경우 대비)
  URL = get_URL(vol_number, last_success + 1)
  response = requests.get(URL)
  if response.status_code != 404:
    whole_req.append(response.text)
    print(vol_number, last_success + 1)  # 확인용

# req 보관용 변수 정의
author_of_article_req = []
genre_of_article_req = []

# parsing 및 텍스트 내용 검색
for page in whole_req:
  page_parsed = bs(page, 'html.parser')
  # print(page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(4) > th')[0].text)  # 확인용
  if page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(4) > th')[0].text == '필자':
    author_of_article_req.append(page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(4) > td')[0].text)
    genre_of_article_req.append(page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(5) > td')[0].text)
  else:
    author_of_article_req.append(None)  # 빈칸
    genre_of_article_req.append(page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(4) > td')[0].text)

# metadata 각각의 list
author_of_article = []
genre_of_article = []

# list에 내용 넣기 
for element in author_of_article_req:
  author_of_article.append(element)
for element in genre_of_article_req:
  genre_of_article.append(element)

10 0
10 1
10 10
10 20
10 30
10 40
10 50
10 60
10 70


# 개벽 본문 + 메타데이터 스크래핑
- 메타데이터 + 본문 모두를 스크래핑하는 최종 코드

In [ ]:
from itertools import chain # 두 번째 for문 0, 1 처리용

# 개벽 전체 텍스트
whole_req = ""
whole_req_list = []

# 개벽 제28호(1부, 2부) 처리용
volumes = list(range(10, 760, 10))
volumes[27:27] = [271, 272]

# 개벽 전체 텍스트 스크래핑
for vol_number in volumes:

  # 호(volume) 내부 텍스트 스크래핑
  for id_number in chain([0, 1], range(10, 9999, 10)):
    URL = get_URL(vol_number, id_number)
    response = requests.get(URL)

    # 404 error 발생 시 다음 volume으로 이동
    if response.status_code == 404:
      if id_number > 10:
        break
    else:
      # URL을 get하여 얻은 response의 text를 리스트에 삽입
      whole_req = whole_req + response.text
      whole_req_list.append(response.text)
    
      # 마지막 텍스트 여부 확인용
      last_success = id_number

    print(vol_number, id_number)  # 확인용

  # 호(volume) 마지막 텍스트 스크래핑용 (1로 끝나는 경우 대비)
  URL = get_URL(vol_number, last_success + 1)
  response = requests.get(URL)
  if response.status_code != 404:
    whole_req = whole_req + response.text
    whole_req_list.append(response.text)
    print(vol_number, last_success + 1)  # 확인용

# 사전 정의 list
txt = []
author_of_article_req = []
genre_of_article_req = []

# parsing 및 텍스트 내용 검색
soup = bs(whole_req, 'html.parser')
elements = soup.select('#cont_view')  # 텍스트 내용 전부

# metadata는 tbody > tr > td 안에 있고, tbody는 4개의 tr(순서대로 잡지명, 발행일, 기사제목, (필자), 기사형태)을 가짐
name_of_megazine_req = soup.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(1) > td')  # 잡지명
date_of_issue_req = soup.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(2) > td')     # 발행일
title_of_article_req = soup.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(3) > td')  # 기사제목

# page 단위 parsing 및 '필자' 정보 유무에 따른 분기 처리
for page in whole_req_list:
  page_parsed = bs(page, 'html.parser')
  # print(page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(4) > th')[0].text)  # 확인용
  if page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(4) > th')[0].text == '필자':
    author_of_article_req.append(page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(4) > td')[0].text)
    genre_of_article_req.append(page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(5) > td')[0].text)
  else:
    author_of_article_req.append(None)  # 빈칸
    genre_of_article_req.append(page_parsed.select('#container > div.cont_view > div.dl_data_pru > table > tbody > tr:nth-of-type(4) > td')[0].text)

# metadata 각각의 list
name_of_megazine = []
date_of_issue = []
title_of_article = []
author_of_article = []
genre_of_article = []

# list에 내용 넣기
for index, element in enumerate(elements, 1):
  txt.append(element.text)
for element in name_of_megazine_req:
  name_of_megazine.append(element.text)
for element in date_of_issue_req:
  date_of_issue.append(element.text)
for element in title_of_article_req:
  title_of_article.append(element.text)
for element in author_of_article_req:
  author_of_article.append(element)
for element in genre_of_article_req:
  genre_of_article.append(element)

10 0
10 1
10 10
10 20
10 30
10 40
10 50
10 60
10 70
10 80
10 90
10 100
10 110
10 120
10 130
10 140
10 150
10 160
10 170
10 180
10 190
10 200
10 210
10 220
10 230
10 240
10 250
10 260
10 270
10 280
10 290
10 300
10 310
10 320
10 330
10 340
10 350
10 360
10 370
10 380
10 390
10 400
10 410
10 420
10 430
10 440
10 450
10 460
10 470
10 480
20 0
20 1
20 10
20 20
20 30
20 40
20 50
20 60
20 70
20 80
20 90
20 100
20 110
20 120
20 130
20 140
20 150
20 160
20 170
20 180
20 190
20 200
20 210
20 220
20 230
20 240
20 250
20 260
20 270
20 280
20 290
20 300
20 310
20 320
20 330
20 340
20 350
20 360
20 370
20 380
20 390
20 400
20 401
30 0
30 1
30 10
30 20
30 30
30 40
30 50
30 60
30 70
30 80
30 90
30 100
30 110
30 120
30 130
30 140
30 150
30 160
30 170
30 180
30 190
30 200
30 210
30 220
30 230
30 240
30 250
30 260
30 270
30 280
30 290
30 300
30 310
30 320
30 330
30 340
30 350
30 360
30 370
30 380
30 390
30 400
30 410
30 420
30 421
40 0
40 1
40 10
40 20
40 30
40 40
40 50
40 60
40 70
40 80
40 90
40 100
40

# xlsx 파일로 변환
- 필요에 맞게 사용할 것

## 본문 변환

In [ ]:
df = pd.DataFrame()
df['text'] = txt

df.to_excel('./개벽_본문.xlsx', sheet_name='Sheet1')

## 메타데이터 변환

In [ ]:
df = pd.DataFrame()
df['name_of_megazine'] = name_of_megazine
df['date_of_issue'] = date_of_issue
df['title_of_article'] = title_of_article
df['author_of_article'] = author_of_article
df['genre_of_article'] = genre_of_article

df.to_excel('./개벽_메타데이터.xlsx', sheet_name='Sheet1')

## 본문 + 메타데이터 변환

In [ ]:
df = pd.DataFrame()
df['name_of_megazine'] = name_of_megazine
df['date_of_issue'] = date_of_issue
df['title_of_article'] = title_of_article
df['author_of_article'] = author_of_article
df['genre_of_article'] = genre_of_article
df['text'] = txt

df.to_excel('./개벽_전문.xlsx', sheet_name='Sheet1')

# tsv, csv로 변환

## tsv

In [ ]:
df = pd.DataFrame()
df['name_of_megazine'] = name_of_megazine
df['date_of_issue'] = date_of_issue
df['title_of_article'] = title_of_article
df['author_of_article'] = author_of_article
df['genre_of_article'] = genre_of_article
df['text'] = txt

df.to_csv('./개벽_전문.tsv', sep='\t', encoding='utf-8')

## csv

In [ ]:
df = pd.DataFrame()
df['name_of_megazine'] = name_of_megazine
df['date_of_issue'] = date_of_issue
df['title_of_article'] = title_of_article
df['author_of_article'] = author_of_article
df['genre_of_article'] = genre_of_article
df['text'] = txt

df.to_csv('./개벽_전문.csv', encoding='utf-8')